In [34]:
import h5py
from freeqdsk import geqdsk

In [33]:
with h5py.File('../Testing/beta1.h5','r') as h5f:
    print(h5f.keys())
    FF = h5f['/FluxFunctions']
    print(FF.keys())
    B = h5f['/Boundaries']
    print(B.keys())
    print(FF["PsiX"][()])
    print(FF["Psi_1D"][()])
    print(h5f['Scalars/simagx'][()])
    print(h5f["Scalars/sibdry"][()])


<KeysViewHDF5 ['Boundaries', 'FluxFunctions', 'Grid', 'Scalars']>
<KeysViewHDF5 ['B2ave_1D', 'Beta_1D', 'Jave_1D', 'PsiX', 'Psi_1D', 'Shear_1D', 'Vol_1D', 'Well_1D', 'dG2dPsi_1D', 'dVdPsi_1D', 'fpol', 'pprime', 'pres', 'qpsi']>
<KeysViewHDF5 ['FCFS', 'LCFS', 'ilim', 'lim']>
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
0.0
0.0


In [36]:
def dipoleq_lim_to_eqdsk(lim):
    import numpy as np
    # lim is a 3D array with shape (nlim, 2, 2)
    # with a start and end point for each limiter
    # in this we will assume that the start is the end of the last
    newlim = np.zeros((lim.shape[0]+1, 2))
    newlim[:-1] = lim[:,0]
    newlim[-1] = lim[-1,1]
    return newlim

    
def dipoleq_to_geqdsk(h5f):
    import numpy as np
    from typing import Dict, Union
    from numpy.typing import ArrayLike
    #gdata = Dict[str, Union[int, float, ArrayLike]]
    gdata = {}

    # 0D values
    # commputational domain
    Grid = h5f['/Grid']
    Flux = h5f['/FluxFunctions']
    eq0d = h5f['/Scalars']

    R = Grid['R'][()]
    Z = Grid['Z'][()]
    gdata['rdim'] = max(R) - min(R)
    gdata['rleft'] = min(R)
    gdata['zdim'] = max(Z) - min(Z)
    gdata['zmid'] = (max(Z) + min(Z))/2
    # reference values
    gdata['rcentr'] = eq0d['rcentr'][()]
    gdata['bcentr'] = eq0d['bcentr'][()]
    # plasma current
    gdata['cpasma'] = eq0d['cpasma'][()]
    gdata['rmagx'] = eq0d['rmagx'][()]
    gdata['zmagx'] = eq0d['zmagx'][()]
    # psi values
    gdata['simagx'] = h5f["/Grid/Psi"].attrs["PsiAxis"][0]
    gdata['sibdry'] = h5f["/Grid/Psi"].attrs["PsiLim"][0]

    # 1D values
    # geqdsk assumes that the radial resolution is the same
    # as the number of X gridpoints
    psiX = Flux['PsiX'][()]
    if (len(psiX) != len(R)):
        # must interpolate
        psiXn = np.linspace(psiX[0], psiX[-1], len(R))
        def regrid(y):
            return np.interp(psiXn, psiX, y)
    else:
        regrid = lambda y: y
    gdata['fpol']       = regrid(Flux['fpol'][()])
    gdata['pres']       = regrid(Flux['pres'][()])
    #gdata['ffprime']   = Flux['ffprime'][()]
    gdata['ffprime']    = regrid(Flux['dG2dPsi_1D'][()]/2)
    gdata['pprime']     = regrid(Flux['pprime'][()])
    gdata['qpsi']       = regrid(Flux['qpsi'][()])

    # 2D values
    gdata['psi'] = Grid['Psi'][()]

    # Boundary values
    lcfs = h5f['/Boundaries/LCFS'][()]
    fcfs = h5f['/Boundaries/FCFS'][()]
    gdata['rbdry'] = lcfs[:,0]
    gdata['zbdry'] = lcfs[:,1]

    olimq = h5f['/Boundaries/lim'][()]
    ilimq = h5f['/Boundaries/ilim'][()]
    olim = dipoleq_lim_to_eqdsk(olimq)
    gdata['rlim'] = olim[:,0]
    gdata['zlim'] = olim[:,1]
    oname = str(h5f.attrs["ONAME"],'utf-8')

    return (gdata, oname)

with h5py.File('../Testing/beta1.h5','r') as h5f:
    gdata, oname = dipoleq_to_geqdsk(h5f)
    ofile = f"{oname}.geqdsk"
    with open(ofile, 'w') as fh:
        geqdsk.write(gdata, fh, label=oname)
        print(f"# {oname:s} written to {ofile}")
    

# beta1 written to beta1.geqdsk
